In [4]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [32]:
def seperate_date_time(x):
    date = x.split(', ')[0]
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [33]:
def createDf(file_data, dt):
    patterns = {
        '12hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24hr':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'user_message': user_message, 'date_time': date_time})
    df[['date','time']] = df['date_time'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message', 'date_time'], inplace=True)
    return df

In [34]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
createDf(f, '12hr')

,date,time,user,message
0,5/9/22,3:01 PM,group_notification,Messages and calls are end-to-end encrypted. N...
1,5/26/22,7:21 PM,Clg Divyansh,Robotics ke notes
2,5/26/22,7:44 PM,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...
3,5/26/22,7:55 PM,Clg Abbas Amjhera,<Media omitted>
4,5/26/22,8:05 PM,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻
...,...,...,...,...
1452,8/18/22,2:01 PM,Clg Charchil,🤚
1453,8/18/22,2:01 PM,+91 87705 37687,✋🏻
1454,8/18/22,2:15 PM,Clg Atman,🙋‍♂️
1455,8/18/22,2:20 PM,Clg Ashvin birla,<Media omitted>


In [35]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '24hr'))

In [31]:
# for i in df['date']:
#     print(i.dt.year)
#     break
df['date'].iloc[0].year

2022